# Installatie
Voordat deze notebook gebruikt kan worden moeten er een aantal programma's geïnstalleerd worden.
Welke en hoe deze programma's geïnstalleerd moeten worden is te vinden in het INSTALL.md bestand.


## Paths

In [19]:
# General
WORKSPACE_PATH = "Tensorflow/workspace"
IMAGE_PATH = WORKSPACE_PATH + "/images"
ANNOTATION_PATH = WORKSPACE_PATH + "/annotations"

# Models
TRAINING_MODELS_PATH = WORKSPACE_PATH + "/models"
EXPORTED_MODELS_PATH = WORKSPACE_PATH + "/exported-models"
PRE_TRAINED_MODELS_PATH = WORKSPACE_PATH + "/pre-trained-models"

# Scripts
SCRIPTS_PATH = "Tensorflow/scripts"
PRE_PROCESSING_SCRIPTS_PATH =  SCRIPTS_PATH + "/preprocessing"
TF_SCRIPTS_PATH = SCRIPTS_PATH + "/tf2"

## Label map

In [2]:
labels = [
    'bottle'
]

In [15]:
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for i in range(0, len(labels)):
        f.write('item { \n')
        f.write('\tid:{}\n'.format(i + 1))
        f.write('\tname:\'{}\'\n'.format(labels[i]))
        f.write('}\n')

## Image XML

In [20]:
!python {PRE_PROCESSING_SCRIPTS_PATH + "/correct_image_xml.py"} -path {IMAGE_PATH}

Successfully changed 0 XML files.


## TF records

In [21]:
!python {SCRIPTS_PATH + '/preprocessing/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/preprocessing/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## Prepare Model

In [22]:
CONFIG_BATCH_SIZE = 4
NEXT_VERSION = 4.0

BASED_ON_PATH = PRE_TRAINED_MODELS_PATH
BASED_ON_NAME = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"

CURRENT_TRAINING_PATH = TRAINING_MODELS_PATH + "/model"

In [23]:
import os
import shutil

if not os.path.exists(CURRENT_TRAINING_PATH):
    os.makedirs(CURRENT_TRAINING_PATH)
    print("Successfully created folder for training.")

    shutil.copy(BASED_ON_PATH + "/" + BASED_ON_NAME + "/pipeline.config", CURRENT_TRAINING_PATH)
    print("Successfully copied the pipeline.config from the based on model.")
else:
    print("Was not able to create folder for training, please check you selected a version which has not been used yet.")

Successfully created folder for training.
Successfully copied the pipeline.config from the based on model.


In [24]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

config = config_util.get_configs_from_pipeline_file(CURRENT_TRAINING_PATH + "/pipeline.config")

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CURRENT_TRAINING_PATH + "/pipeline.config", "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

# Change CONFIG information
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = CONFIG_BATCH_SIZE
pipeline_config.train_config.fine_tune_checkpoint = BASED_ON_PATH + "/" + BASED_ON_NAME + "/checkpoint/ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CURRENT_TRAINING_PATH + "/pipeline.config", "wb") as f:
    f.write(config_text)

config = config_util.get_configs_from_pipeline_file(CURRENT_TRAINING_PATH + "/pipeline.config")

In [25]:
config

{'model': ssd {
   num_classes: 1
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

# Train model

In [26]:
STEPS = 40000

print(f"python {TF_SCRIPTS_PATH}/model_main_tf2.py -model_dir={CURRENT_TRAINING_PATH} --pipeline_config_path={CURRENT_TRAINING_PATH}/pipeline.config --num_train_steps={STEPS}")

python Tensorflow/scripts/tf2/model_main_tf2.py -model_dir=Tensorflow/workspace/models/model --pipeline_config_path=Tensorflow/workspace/models/model/pipeline.config --num_train_steps=400000


# Export model

In [11]:
print(f"python {TF_SCRIPTS_PATH}/exporter_main_v2.py --input_type image_tensor --pipeline_config_path {CURRENT_TRAINING_PATH}/pipeline.config --trained_checkpoint_dir {CURRENT_TRAINING_PATH} --output_directory {EXPORTED_MODELS_PATH}/detection_v{str(NEXT_VERSION)}")

python Tensorflow/scripts/tf2/exporter_main_v2.py --input_type image_tensor --pipeline_config_path Tensorflow/workspace/models/model/pipeline.config --trained_checkpoint_dir Tensorflow/workspace/models/model --output_directory Tensorflow/workspace/exported-models/detection_v4.0
